# Preprocessing and Exploratory analysis of regression dataset downloaded from kaggle

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import plotly.express as px
from sklearn import metrics
from pandas.tseries.holiday import (
    AbstractHolidayCalendar, Holiday, DateOffset, EasterMonday, GoodFriday, MO, next_monday, next_monday_or_tuesday
)
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from pandas.tseries.holiday import AbstractHolidayCalendar, nearest_workday, Easter
from pandas.tseries.offsets import  CustomBusinessDay

## Preprocessing and One-Hot Encoding

In [95]:
class MyCustomCalendar(AbstractHolidayCalendar):
    rules = [Holiday("christmasweek", month=12, day=24),Holiday("christmasweek", month=12, day=25), Holiday("christmasweek", month=12, day=26), Holiday("christmasweek", month=12, day=27),
             Holiday("christmasweek", month=12, day=28), Holiday("christmasweek", month=12, day=29), Holiday("christmasweek", month=12, day=30), Holiday("christmasweek", month=12, day=31), Holiday("christmasweek", month=1, day=1) , Holiday("easterweek",month=1 , day=1, offset= [Easter()])]

cal = MyCustomCalendar()
holidays = cal.holidays(start='2015-01-01', end='2018-12-31')
holidays

DatetimeIndex(['2015-01-01', '2015-04-05', '2015-12-24', '2015-12-25',
               '2015-12-26', '2015-12-27', '2015-12-28', '2015-12-29',
               '2015-12-30', '2015-12-31', '2016-01-01', '2016-03-27',
               '2016-12-24', '2016-12-25', '2016-12-26', '2016-12-27',
               '2016-12-28', '2016-12-29', '2016-12-30', '2016-12-31',
               '2017-01-01', '2017-04-16', '2017-12-24', '2017-12-25',
               '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29',
               '2017-12-30', '2017-12-31', '2018-01-01', '2018-04-01',
               '2018-12-24', '2018-12-25', '2018-12-26', '2018-12-27',
               '2018-12-28', '2018-12-29', '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns]', freq=None)

In [96]:
original_train = pd.read_csv('train.csv')
original_train['Week_Number'] = pd.to_datetime(original_train['date']).dt.isocalendar().week
original_train['Month_Number'] = pd.to_datetime(original_train['date']).dt.month
original_train['Year_Number'] = pd.to_datetime(original_train['date']).dt.year
original_train['Day_Number'] = pd.to_datetime(original_train['date']).dt.weekday
#original_train['Christmas_Week'] =
original_train['Special_Day'] = pd.to_datetime(original_train['date']).isin(holidays)
original_train['Weekend'] = original_train['Day_Number'] >= 5
original_train


,row_id,date,country,store,product,num_sold,Week_Number,Month_Number,Year_Number,Day_Number,Special_Day,Weekend
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329,1,1,2015,3,True,False
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520,1,1,2015,3,True,False
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146,1,1,2015,3,True,False
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572,1,1,2015,3,True,False
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911,1,1,2015,3,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
26293,26293,2018-12-31,Sweden,KaggleMart,Kaggle Hat,823,1,12,2018,0,True,False
26294,26294,2018-12-31,Sweden,KaggleMart,Kaggle Sticker,250,1,12,2018,0,True,False
26295,26295,2018-12-31,Sweden,KaggleRama,Kaggle Mug,1004,1,12,2018,0,True,False
26296,26296,2018-12-31,Sweden,KaggleRama,Kaggle Hat,1441,1,12,2018,0,True,False


In [97]:
def one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1) #.drop([each], axis=1)
    return df
ohe_train = one_hot(original_train,['Year_Number', 'Week_Number', 'Month_Number', 'product', 'store', 'country', 'Weekend','Special_Day'])
ohe_train = ohe_train.drop(['Week_Number', 'Month_Number', 'product', 'store', 'country', 'date', 'row_id', 'Day_Number', 'Weekend','Special_Day' ], axis=1)
ohe_train = ohe_train.drop(['Weekend_False'], axis=1)
ohe_train = ohe_train.drop(['Special_Day_False'], axis=1)
new_train = ohe_train[ohe_train['Year_Number'] < 2018]
new_test = ohe_train[ohe_train['Year_Number'] == 2018]
new_train = new_train.drop(['Year_Number'],axis=1)
new_test = new_test.drop(['Year_Number'],axis=1)
new_train

,num_sold,Year_Number_2015,Year_Number_2016,Year_Number_2017,Year_Number_2018,Week_Number_1,Week_Number_2,Week_Number_3,Week_Number_4,Week_Number_5,...,product_Kaggle Hat,product_Kaggle Mug,product_Kaggle Sticker,store_KaggleMart,store_KaggleRama,country_Finland,country_Norway,country_Sweden,Weekend_True,Special_Day_True
0,329,1,0,0,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
1,520,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,1,0,0,0,1
2,146,1,0,0,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,572,1,0,0,0,1,0,0,0,0,...,0,1,0,0,1,1,0,0,0,1
4,911,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19723,1037,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,1,1
19724,290,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,1,1
19725,1188,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,1,1
19726,1781,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,1,1


In [98]:
train_split, validation_split = train_test_split(new_train, test_size=0.2, random_state=123)
type(train_split)

pandas.core.frame.DataFrame

In [99]:
x = train_split.drop('num_sold', axis=1)
y = train_split['num_sold']

regr = linear_model.LinearRegression().fit(x,y)
regr.score(x,y)


0.8332764988243974

In [100]:
regr.coef_
coef_df = pd.DataFrame(zip(x.columns,regr.coef_))
coef_df.columns=['feature_name', 'coeff']

In [101]:
px.bar(coef_df, y='feature_name', x='coeff', orientation='h').show()

In [102]:
validation_x = validation_split.drop(['num_sold'], axis=1)
validation_y = validation_split['num_sold']

In [103]:
regr.predict(validation_x)

array([382.   , 152.   , 298.375, ..., 191.5  , 662.   , 696.75 ])

In [104]:
pred_df = pd.DataFrame(zip(regr.predict(validation_x), validation_y))
pred_df.columns=['predicted_values' , 'true_values']
pred_df

,predicted_values,true_values
0,382.000,430
1,152.000,186
2,298.375,185
3,227.000,170
4,432.500,386
...,...,...
3941,647.250,675
3942,555.125,463
3943,191.500,206
3944,662.000,718


In [105]:
metrics.r2_score(pred_df['true_values'], pred_df['predicted_values'])

0.8225481311427653

In [106]:
px.scatter(x = pred_df['true_values'], y = pred_df['predicted_values']).show()